In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
task_names = [
    "COLA", 
#     "SST2", 
#     "MNLI",
#     "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]

FILENAME = "COLA_tasks_and_payloads"

In [3]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# Create tasks and payloads
tasks, payloads = create_tasks_and_payloads(
    task_names,
#     max_datapoints=1000,
    dl_kwargs={"batch_size": 16}
)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Using random seed: 32964
Loading COLA Dataset



Added label_set with 8550 labels for task COLA:ends_with_question to payload COLA_train.
Added label_set with 1042 labels for task COLA:ends_with_question to payload COLA_valid.
Added label_set with 1063 labels for task COLA:ends_with_question to payload COLA_test.
CPU times: user 48.8 s, sys: 1.89 s, total: 50.7 s
Wall time: 26.5 s


In [4]:
# %%time
# # Pickle tasks and payloads
# import dill
# with open("FILENAME", 'wb') as f:
#     dill.dump((tasks, payloads), f)

In [ ]:
# %%time
# # Unpickle tasks and payloads
# import dill
# with open("FILENAME", 'rb') as f:
#     (tasks, payloads) = dill.load(f)

In [ ]:
pprint(tasks)
pprint(payloads)

[ClassificationTask(name=COLA, loss_multiplier=1.0),
 ClassificationTask(name=COLA:ends_with_question, loss_multiplier=1.0),
 ClassificationTask(name=COLA:ends_with_question, loss_multiplier=1.0),
 ClassificationTask(name=COLA:ends_with_question, loss_multiplier=1.0)]
[Payload(COLA_train: tasks=[COLA,COLA:ends_with_question], split=train),
 Payload(COLA_valid: tasks=[COLA,COLA:ends_with_question], split=valid),
 Payload(COLA_test: tasks=[COLA,COLA:ends_with_question], split=test)]


In [ ]:
SEED = 1

In [ ]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, seed=SEED, verbose=False)

In [ ]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

In [ ]:
trainer.train_model(
    model,
    payloads,
    checkpoint_metric="COLA/COLA_valid/matthews_corr",
    checkpoint_metric_mode="max",
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    log_every=0.05,
    score_every=0.1,
    n_epochs=8,
    progress_bar=True
)

Beginning train loop.
Expecting a total of approximately 8560 examples and 535 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/bradenjh/metal/logs/2019_03_14/19_45_27/config.json


/dfs/scratch0/bradenjh/metal/metal/logging/tensorboard.py:26: UserWarning: At iteration 27, metric COLA:ends_with_question/train/loss had value None!
  warnings.warn(f"At iteration {i}, metric {name} had value None!")


[0.05 epo]: COLA:[train/loss=6.28e-01] COLA:ends_with_question:[train/loss=None] model:[train/loss=6.28e-01, train/lr=5.00e-05]
1042 1042
27 1042
[0.10 epo]: COLA:[train/loss=5.77e-01, COLA_valid/accuracy=7.20e-01, COLA_valid/matthews_corr=2.36e-01] COLA:ends_with_question:[train/loss=3.12e-01, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=5.76e-01, train/lr=5.00e-05, valid/glue=2.36e-01]


/dfs/scratch0/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/dfs/scratch0/bradenjh/metal/metal/mmtl/trainer.py:471: UserWarning: You requested glue score but have fewer than 9 tasks. Be aware.
  warnings.warn(msg)


[0.15 epo]: COLA:[train/loss=5.07e-01] COLA:ends_with_question:[train/loss=4.99e-01] model:[train/loss=5.07e-01, train/lr=5.00e-05]
1042 1042
27 1042
[0.20 epo]: COLA:[train/loss=5.10e-01, COLA_valid/accuracy=7.68e-01, COLA_valid/matthews_corr=4.10e-01] COLA:ends_with_question:[train/loss=4.99e-01, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=5.10e-01, train/lr=5.00e-05, valid/glue=4.10e-01]
[0.25 epo]: COLA:[train/loss=4.97e-01] COLA:ends_with_question:[train/loss=1.71e-01] model:[train/loss=4.96e-01, train/lr=5.00e-05]
1042 1042
27 1042
[0.30 epo]: COLA:[train/loss=5.17e-01, COLA_valid/accuracy=7.67e-01, COLA_valid/matthews_corr=4.14e-01] COLA:ends_with_question:[train/loss=1.93e-01, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=5.15e-01, train/lr=5.00e-05, valid/glue=4.14e-01]
[0.35 epo]: COLA:[train/loss=4.80e-01] COLA:ends_with_question:[train/loss=2.07e-01] model:[train/loss=4.79e-01, train/lr=5.00e-05]
10

...

1042 1042
27 1042
[2.02 epo]: COLA:[train/loss=2.68e-01, COLA_valid/accuracy=8.23e-01, COLA_valid/matthews_corr=5.65e-01] COLA:ends_with_question:[train/loss=1.21e-02, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=2.68e-01, train/lr=5.00e-05, valid/glue=5.65e-01]
[2.07 epo]: COLA:[train/loss=2.12e-01] COLA:ends_with_question:[train/loss=1.42e-02] model:[train/loss=2.11e-01, train/lr=5.00e-05]
1042 1042
27 1042
[2.12 epo]: COLA:[train/loss=1.84e-01, COLA_valid/accuracy=8.40e-01, COLA_valid/matthews_corr=6.11e-01] COLA:ends_with_question:[train/loss=8.12e-03, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=1.83e-01, train/lr=5.00e-05, valid/glue=6.11e-01]
[2.17 epo]: COLA:[train/loss=2.67e-01] COLA:ends_with_question:[train/loss=9.92e-03] model:[train/loss=2.66e-01, train/lr=5.00e-05]
1042 1042
27 1042
[2.22 epo]: COLA:[train/loss=2.06e-01, COLA_valid/accuracy=8.32e-01, COLA_valid/matthews_corr=5.89e-01] COLA:ends_wi

1042 1042
27 1042
[3.03 epo]: COLA:[train/loss=1.54e-01, COLA_valid/accuracy=8.10e-01, COLA_valid/matthews_corr=5.29e-01] COLA:ends_with_question:[train/loss=5.24e-03, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=1.53e-01, train/lr=5.00e-05, valid/glue=5.29e-01]
[3.08 epo]: COLA:[train/loss=8.43e-02] COLA:ends_with_question:[train/loss=2.65e-03] model:[train/loss=8.41e-02, train/lr=5.00e-05]
1042 1042
27 1042
[3.13 epo]: COLA:[train/loss=2.41e-01, COLA_valid/accuracy=8.34e-01, COLA_valid/matthews_corr=5.93e-01] COLA:ends_with_question:[train/loss=2.65e-03, COLA_valid/accuracy=4.07e-01, COLA_valid/matthews_corr=-2.19e-01] model:[train/loss=2.41e-01, train/lr=5.00e-05, valid/glue=5.93e-01]
[3.18 epo]: COLA:[train/loss=1.51e-01] COLA:ends_with_question:[train/loss=5.26e-03] model:[train/loss=1.51e-01, train/lr=5.00e-05]
1042 1042
27 1042
[3.23 epo]: COLA:[train/loss=1.40e-01, COLA_valid/accuracy=8.21e-01, COLA_valid/matthews_corr=5.58e-01] COLA:ends_w

1042 1042
27 1042
[4.04 epo]: COLA:[train/loss=1.25e-01, COLA_valid/accuracy=8.18e-01, COLA_valid/matthews_corr=5.50e-01] COLA:ends_with_question:[train/loss=2.18e-03, COLA_valid/accuracy=4.07e-01, COLA_valid/matthews_corr=-2.19e-01] model:[train/loss=1.25e-01, train/lr=5.00e-05, valid/glue=5.50e-01]
[4.09 epo]: COLA:[train/loss=9.65e-02] COLA:ends_with_question:[train/loss=9.86e-04] model:[train/loss=9.63e-02, train/lr=5.00e-05]
1042 1042
27 1042
[4.14 epo]: COLA:[train/loss=1.34e-01, COLA_valid/accuracy=8.18e-01, COLA_valid/matthews_corr=5.49e-01] COLA:ends_with_question:[train/loss=1.48e-03, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=1.33e-01, train/lr=5.00e-05, valid/glue=5.49e-01]
[4.19 epo]: COLA:[train/loss=8.67e-02] COLA:ends_with_question:[train/loss=1.48e-03] model:[train/loss=8.67e-02, train/lr=5.00e-05]
1042 1042
27 1042
[4.24 epo]: COLA:[train/loss=9.83e-02, COLA_valid/accuracy=8.18e-01, COLA_valid/matthews_corr=5.49e-01] COLA:ends_w

1042 1042
27 1042
[5.05 epo]: COLA:[train/loss=6.37e-02, COLA_valid/accuracy=8.31e-01, COLA_valid/matthews_corr=5.86e-01] COLA:ends_with_question:[train/loss=3.47e-03, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=6.37e-02, train/lr=5.00e-05, valid/glue=5.86e-01]
[5.10 epo]: COLA:[train/loss=1.30e-01] COLA:ends_with_question:[train/loss=1.23e-03] model:[train/loss=1.30e-01, train/lr=5.00e-05]
1042 1042
27 1042
[5.15 epo]: COLA:[train/loss=6.27e-02, COLA_valid/accuracy=8.30e-01, COLA_valid/matthews_corr=5.83e-01] COLA:ends_with_question:[train/loss=3.95e-03, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=6.25e-02, train/lr=5.00e-05, valid/glue=5.83e-01]
[5.20 epo]: COLA:[train/loss=1.04e-01] COLA:ends_with_question:[train/loss=5.98e-04] model:[train/loss=1.03e-01, train/lr=5.00e-05]
1042 1042
27 1042
[5.25 epo]: COLA:[train/loss=6.74e-02, COLA_valid/accuracy=8.27e-01, COLA_valid/matthews_corr=5.79e-01] COLA:ends_wi

[6.01 epo]: COLA:[train/loss=1.35e-01] COLA:ends_with_question:[train/loss=1.96e-03] model:[train/loss=1.35e-01, train/lr=5.00e-05]
1042 1042
27 1042
[6.06 epo]: COLA:[train/loss=4.30e-02, COLA_valid/accuracy=8.20e-01, COLA_valid/matthews_corr=5.55e-01] COLA:ends_with_question:[train/loss=7.86e-04, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=4.28e-02, train/lr=5.00e-05, valid/glue=5.55e-01]
[6.11 epo]: COLA:[train/loss=6.84e-02] COLA:ends_with_question:[train/loss=5.35e-04] model:[train/loss=6.80e-02, train/lr=5.00e-05]
1042 1042
27 1042
[6.16 epo]: COLA:[train/loss=2.38e-02, COLA_valid/accuracy=8.25e-01, COLA_valid/matthews_corr=5.70e-01] COLA:ends_with_question:[train/loss=8.85e-04, COLA_valid/accuracy=4.44e-01, COLA_valid/matthews_corr=0.00e+00] model:[train/loss=2.38e-02, train/lr=5.00e-05, valid/glue=5.70e-01]
[6.21 epo]: COLA:[train/loss=3.45e-02] COLA:ends_with_question:[train/loss=8.85e-04] model:[train/loss=3.45e-02, train/lr=5.00e-05]
10

In [ ]:
print("WE WIN!")